## Interpolating the data

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import RBFInterpolator, Rbf
from numpy.linalg import LinAlgError

In [ ]:
# 1. Load data
df = pd.read_csv('../Data/CleanedData.csv')  # replace with your CSV file path
df.head()

,helical_angle,helical_layer_count,hoop_layer_count,bust_pressure,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers
0,0,0,0.0,277,0.0,0.0,57.5,0.33,240.3,317,6,130,0
1,0,0,0.0,273,0.0,0.0,57.5,0.33,240.3,317,6,130,0
2,0,0,0.0,275,0.0,0.0,57.5,0.33,240.3,317,6,130,0
3,14,3,3.0,519,1082.0,108.3,57.5,0.33,240.3,317,6,130,0
4,14,3,3.0,383,1082.0,108.3,57.5,0.33,240.3,317,6,130,0


In [4]:
features = [
    'tensile_str', 'e1_gpa', 'youngs_modulus', 'poision_ratio',
    'yeild_strength', 'diameter', 'ult_tensile_strength',
    'helical_angle', 'helical_layer_count', 'hoop_layer_count',
    'liner_thickness', 'doily_layers'
]
target = 'bust_pressure'

In [5]:
X = df[features].values      # shape (49, 12)
y = df['bust_pressure'].values  # shape (49,)
X.shape, y.shape

((49, 12), (49,))

In [6]:
rbf_interp = None
smoothing = 0.0
for trial in range(6):  # try smoothing = 0.0, 1e-2, 1e-3, ...
    try:
        rbf_interp = RBFInterpolator(X, y, kernel='linear', smoothing=smoothing)
        print(f"✅ RBFInterpolator succeeded with smoothing={smoothing}")
        break
    except LinAlgError:
        smoothing = 10**(-trial-2)  # 1e-2, 1e-3, ..., 1e-7
else:
    print("⚠️ RBFInterpolator failed — using fallback Rbf model.")
    rbf_interp = None


✅ RBFInterpolator succeeded with smoothing=0.01


In [7]:
# 4. If still failed, fallback to Rbf (older but more tolerant)
use_old = False
if rbf_interp is None:
    rbf_old = Rbf(
        *[X[:, i] for i in range(X.shape[1])],
        y,
        function='multiquadric',
        epsilon=1.0,
        smooth=1e-2
    )
    use_old = True


In [8]:
new_samples = np.array([
    [400.0, 5.0,   200.0, 0.3, 350.0, 50.0, 500.0, 30.0, 10, 10, 2.0, 5],
    [450.0, 5.5,   210.0, 0.29, 360.0, 55.0, 520.0, 35.0, 12,  9, 1.8, 4]
])


In [9]:
if not use_old:
    y_pred = rbf_interp(new_samples)
else:
    y_pred = rbf_old(*[new_samples[:, i] for i in range(new_samples.shape[1])])

In [10]:
# 7. Display results in a DataFrame
result_df = pd.DataFrame(new_samples, columns=features)
result_df['predicted_burst_pressure'] = y_pred
print("\n🧪 Interpolation Results:\n")
print(result_df.to_string(index=False))


🧪 Interpolation Results:

 tensile_str  e1_gpa  youngs_modulus  poision_ratio  yeild_strength  diameter  ult_tensile_strength  helical_angle  helical_layer_count  hoop_layer_count  liner_thickness  doily_layers  predicted_burst_pressure
       400.0     5.0           200.0           0.30           350.0      50.0                 500.0           30.0                 10.0              10.0              2.0           5.0                840.716344
       450.0     5.5           210.0           0.29           360.0      55.0                 520.0           35.0                 12.0               9.0              1.8           4.0                873.547691


In [11]:
df.sample(10)  # Display the original DataFrame for reference

,helical_angle,helical_layer_count,hoop_layer_count,bust_pressure,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers
27,14,9,7.0,1235,1100.0,95.60,57.5,0.33,240.3,317,6,130,17
18,14,12,11.0,1050,1100.0,95.60,57.5,0.33,240.3,317,6,130,0
26,14,9,7.0,1211,1100.0,95.60,57.5,0.33,240.3,317,6,130,17
4,14,3,3.0,383,1082.0,108.30,57.5,0.33,240.3,317,6,130,0
48,0,0,0.0,692,0.0,0.00,192.0,0.28,235.0,360,4,132,0
39,11,3,3.0,848,609.8,27.93,192.0,0.28,235.0,360,4,132,0
9,14,5,3.0,500,1082.0,108.30,57.5,0.33,240.3,317,6,130,0
16,14,9,9.0,745,1082.0,108.30,57.5,0.33,240.3,317,6,130,0
46,0,0,0.0,657,0.0,0.00,192.0,0.28,235.0,360,4,132,0
24,14,9,7.0,1170,1100.0,95.60,57.5,0.33,240.3,317,6,130,11


In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
helical_angle,49.0,11.795918,4.587027,0.00,11.00,14.00,14.00,14.00
helical_layer_count,49.0,6.632653,3.930086,0.00,3.00,9.00,9.00,15.00
hoop_layer_count,49.0,5.836735,3.202193,0.00,3.00,7.00,7.50,11.00
bust_pressure,49.0,894.326531,328.761590,273.00,657.00,923.00,1129.00,1518.00
tensile_str,49.0,879.763265,376.677403,0.00,609.80,1082.00,1100.00,1100.00
e1_gpa,49.0,76.733469,39.908225,0.00,27.93,95.60,108.30,108.30
youngs_modulus,49.0,87.693878,56.701137,57.50,57.50,57.50,57.50,192.00
poision_ratio,49.0,0.318776,0.021078,0.28,0.33,0.33,0.33,0.33
yeild_strength,49.0,239.110204,2.234320,235.00,240.30,240.30,240.30,240.30
ult_tensile_strength,49.0,326.653061,18.127501,317.00,317.00,317.00,317.00,360.00


## Generating synthetic samples

In [13]:
from dependencies.constants import *

feature_ranges = {
    'helical_angle': (HELICAL_ANGLE_LB, HELICAL_ANGLE_UB),         # degrees
    'helical_layer_count': (HELICAL_LAYER_LB, HELICAL_LAYER_UB),        # integer
    'hoop_layer_count': (HOOP_LAYER_LB, HOOP_LAYER_UB),           # integer
    'tensile_str': (TENSILE_STRENGTH_LB, TENSILE_STRENGTH_UB),         # MPa
    'e1_gpa': (E1_LB, E1_UB),                  # GPa
    'youngs_modulus': (YOUNGS_MODULUS_LB, YOUNGS_MODULUS_UB),      # GPa
    'poision_ratio': (POISSONS_RATIO_LB, POISSONS_RATIO_UB),         # unitless
    'yeild_strength': (YEILD_STRENGTH_LB, YEILD_STRENGTH_UB),      # MPa
    'ult_tensile_strength': (ULT_STRENGTH_LB, ULT_STRENGTH_UB),# MPa
    'liner_thickness': (LINER_THICKNESS_LB, LINER_THICKNESS_UB),         # mm
    'diameter': (DIAMETER_LB, DIAMETER_UB),              # mm
    'doily_layers': (DOLIY_LB, DOLIY_UB)                 # integer
}

features = list(feature_ranges.keys())
features

['helical_angle',
 'helical_layer_count',
 'hoop_layer_count',
 'tensile_str',
 'e1_gpa',
 'youngs_modulus',
 'poision_ratio',
 'yeild_strength',
 'ult_tensile_strength',
 'liner_thickness',
 'diameter',
 'doily_layers']

In [18]:
# Generate N samples
N = 300  # You can change this
synthetic_samples = []

np.random.seed(123)  # For reproducibility

for _ in range(N):
    sample = []
    for feat in features:
        low, high = feature_ranges[feat]
        if isinstance(low, int) and isinstance(high, int):
            sample.append(np.random.randint(low, high+1))
        else:
            sample.append(np.random.uniform(low, high))
    synthetic_samples.append(sample)

X_synth = np.array(synthetic_samples)


In [19]:
if not use_old:
    y_synth = rbf_interp(X_synth)
else:
    y_synth = rbf_old(*[X_synth[:, i] for i in range(X_synth.shape[1])])

In [ ]:
df_synth = pd.DataFrame(X_synth, columns=features)
df_synth['bust_pressure'] = y_synth
df_synth = df_synth.round(2)
df_synth.to_csv('../Data/InterpolatedData.csv', index=False)

df_synth.shape

(300, 13)

In [21]:
df_synth.head(10)

,helical_angle,helical_layer_count,hoop_layer_count,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers,bust_pressure
0,23.0,5.0,13.0,698.0,101.0,72.0,0.32,231.0,342.0,0.52,135.0,16.0,621.62
1,10.0,3.0,10.0,939.0,109.0,91.0,0.29,234.0,359.0,0.44,122.0,14.0,642.96
2,12.0,7.0,9.0,1045.0,95.0,62.0,0.32,250.0,348.0,0.46,142.0,9.0,650.75
3,11.0,15.0,9.0,987.0,105.0,124.0,0.29,232.0,368.0,0.50,133.0,25.0,649.61
4,14.0,14.0,12.0,853.0,107.0,85.0,0.30,243.0,361.0,0.51,126.0,21.0,637.21
5,30.0,10.0,12.0,1038.0,102.0,56.0,0.30,235.0,365.0,0.50,140.0,24.0,659.32
6,22.0,5.0,2.0,601.0,98.0,99.0,0.29,246.0,366.0,0.47,142.0,10.0,611.01
7,13.0,14.0,6.0,1073.0,102.0,58.0,0.33,249.0,366.0,0.51,129.0,21.0,656.25
8,16.0,6.0,10.0,798.0,107.0,109.0,0.31,247.0,356.0,0.43,140.0,10.0,629.85
9,11.0,3.0,6.0,960.0,108.0,141.0,0.28,238.0,335.0,0.46,120.0,10.0,640.99
